In [8]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

In [9]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Set things up

In [12]:
import caveclient as cc
import importlib.metadata

print(f"CAVEclient version: v{cc.__version__} , v{importlib.metadata.version('CAVEclient')}")

datastack_name = "minnie65_phase3_v1"

client = cc.CAVEclient(datastack_name)
client.materialize.version = 1078

CAVEclient version: v7.6.1 , v7.6.1


## Pick a test server:
* **localhost:5000 — Test SkeletonService on the local machine, say via the VS Code Debugger**
* **ltv5 — The SkeletonService on the test cluster**
* **minniev6 — Test SkeletonService "in the wild"**

In [19]:
# server_address = "https://localhost:5000"
server_address = "https://ltv5.microns-daf.com"
# server_address = "https://minniev6.microns-daf.com"

skclient = cc.skeletonservice.SkeletonClient(server_address, datastack_name, over_client=client, verify=False)

In [42]:
bulk_rids = [864691135463611454, 864691135687456480]
larger_bulk_rids = bulk_rids * 6  # Twelve rids will exceed the ten-rid limit of get_bulk_skeletons()
single_rid = bulk_rids[0]
sample_refusal_list_rid = 11223344556677889900
sample_invalid_node_rid = 864691135687000000
sample_supervoxel_rid = 88891049011371731
skvn = 4

# Delete the test rid files from the bucket so we can test regenerating them from scratch

In [24]:
from cloudfiles import CloudFiles

bucket = None
if "localhost" in server_address or "ltv" in server_address:
    bucket = f"gs://minnie65_skeletons/ltv/{datastack_name}/{skvn}"
elif "minnie" in server_address:
    bucket = f"gs://minnie65_skeletons/{datastack_name}/{skvn}"
print(f"Testing bucket: {bucket}")

cf = CloudFiles(bucket)
for rid in bulk_rids:
    for output_format in ["h5", "flatdict", "swccompressed"]:
        filename = f"skeleton__v{skvn}__rid-{rid}__ds-{datastack_name}__res-1x1x1__cs-True__cr-7500.{output_format}.gz"
        print(filename)
        print(cf.exists(filename))

Testing bucket: gs://minnie65_skeletons/ltv/minnie65_phase3_v1/4
skeleton__v4__rid-864691135463611454__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
skeleton__v4__rid-864691135463611454__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
True
skeleton__v4__rid-864691135463611454__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
True
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
False


In [25]:
from cloudfiles import CloudFiles

cf = CloudFiles(bucket)
for rid in bulk_rids:
    for output_format in ["h5", "flatdict", "swccompressed"]:
        filename = f"skeleton__v{skvn}__rid-{rid}__ds-{datastack_name}__res-1x1x1__cs-True__cr-7500.{output_format}.gz"
        print(filename)
        print(cf.exists(filename))
        cf.delete(filename)
        print(cf.exists(filename))

skeleton__v4__rid-864691135463611454__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
False
skeleton__v4__rid-864691135463611454__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
True
False
skeleton__v4__rid-864691135463611454__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
True
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
False
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
False
False


## Metadata tests

In [26]:
precomputed_skeleton_info = skclient.get_precomputed_skeleton_info(skvn=skvn)
print(precomputed_skeleton_info)
assert precomputed_skeleton_info == {
    '@type': 'neuroglancer_skeletons',
    'transform': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    'vertex_attributes': [
        {'id': 'radius', 'data_type': 'float32', 'num_components': 1},
        {'id': 'compartment', 'data_type': 'uint8', 'num_components': 1}
    ]
}
print("TEST PASSED")

{'@type': 'neuroglancer_skeletons', 'transform': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], 'vertex_attributes': [{'id': 'radius', 'data_type': 'float32', 'num_components': 1}, {'id': 'compartment', 'data_type': 'uint8', 'num_components': 1}]}
TEST PASSED


In [31]:
import json

rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids)
# print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: False,
    bulk_rids[1]: False
}
print("TEST PASSED")

rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids, log_warning=False)
# print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: False,
    bulk_rids[1]: False
}
print("TEST PASSED")

rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids, verbose_level=1)
# print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: False,
    bulk_rids[1]: False
}
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


TEST PASSED


INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


TEST PASSED
TEST PASSED


In [32]:
import json

cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids)
# print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 0,
    "files": []
}
print("TEST PASSED")

cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids, log_warning=False)
# print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 0,
    "files": []
}
print("TEST PASSED")

cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids, verbose_level=1)
# print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 0,
    "files": []
}
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


TEST PASSED


INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


TEST PASSED
TEST PASSED


# Invalid skeleton request tests

In [35]:
import requests

err = None
try:
    sk = skclient.get_skeleton(sample_refusal_list_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
except requests.HTTPError as e:
    print(e)
    assert e.response.text == '{\n    "Error": "Problematic root id: ' + str(sample_refusal_list_rid) + ' is in the refusal list"\n}\n'
print("TEST PASSED")

err = None
try:
    sk = skclient.get_skeleton(sample_invalid_node_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
except requests.HTTPError as e:
    print(e)
    assert e.response.text == '{\n    "Error": "Invalid root id: ' + str(sample_invalid_node_rid) + ' (perhaps it doesn\'t exist; the error is unclear)"\n}\n'
print("TEST PASSED")

err = None
try:
    sk = skclient.get_skeleton(sample_supervoxel_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
except requests.HTTPError as e:
    print(e)
    assert e.response.text == '{\n    "Error": "Invalid root id: ' + str(sample_supervoxel_rid) + ' (perhaps this is an id corresponding to a different level of the PCG, e.g., a supervoxel id)"\n}\n'
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.9
INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.9


400 Client Error: BAD REQUEST for url: https://ltv5.microns-daf.com/skeletoncache/api/v1/minnie65_phase3_v1/async/get_skeleton/4/11223344556677889900/flatdict?verbose_level=1 content: b'{\n    "Error": "Problematic root id: 11223344556677889900 is in the refusal list"\n}\n'
TEST PASSED


INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.9


400 Client Error: BAD REQUEST for url: https://ltv5.microns-daf.com/skeletoncache/api/v1/minnie65_phase3_v1/async/get_skeleton/4/864691135687000000/flatdict?verbose_level=1 content: b'{\n    "Error": "Invalid root id: 864691135687000000 (perhaps it doesn\'t exist; the error is unclear)"\n}\n'
TEST PASSED
400 Client Error: BAD REQUEST for url: https://ltv5.microns-daf.com/skeletoncache/api/v1/minnie65_phase3_v1/async/get_skeleton/4/88891049011371731/flatdict?verbose_level=1 content: b'{\n    "Error": "Invalid root id: 88891049011371731 (perhaps this is an id corresponding to a different level of the PCG, e.g., a supervoxel id)"\n}\n'
TEST PASSED


# Skeleton request tests

In [36]:
%%time
sk = skclient.get_skeleton(single_rid, datastack_name, skeleton_version=skvn, output_format='dict')
# display(sk)
assert sk is not None and isinstance(sk, dict)
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.9
INFO:root:get_skeleton() response contains content of size 67466 bytes


TEST PASSED
CPU times: user 58.5 ms, sys: 20.9 ms, total: 79.4 ms
Wall time: 22.9 s


In [37]:
%%time
sk = skclient.get_skeleton(single_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
# display(sk)
assert sk is not None and isinstance(sk, dict)
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.9
INFO:root:get_skeleton() response contains content of size 67466 bytes


TEST PASSED
CPU times: user 16.9 ms, sys: 4.8 ms, total: 21.7 ms
Wall time: 2.33 s


In [38]:
%%time
import pandas as pd

sk = skclient.get_skeleton(single_rid, datastack_name, skeleton_version=skvn, output_format='swc', verbose_level=1)
# display(sk)
assert sk is not None and isinstance(sk, pd.DataFrame)
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.9
INFO:root:get_skeleton() response contains content of size 28205 bytes


TEST PASSED
CPU times: user 29.9 ms, sys: 7.9 ms, total: 37.8 ms
Wall time: 1.59 s


### Inspect the cache after generating new skeletons

In [39]:
rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids)
print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: True,
    bulk_rids[1]: False
}
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


{
    "864691135463611454": true,
    "864691135687456480": false
}
TEST PASSED


In [40]:
import json
cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids)
print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 1,
    "files": [
        f"skeleton__v4__rid-{bulk_rids[0]}__ds-{datastack_name}__res-1x1x1__cs-True__cr-7500.h5.gz"
    ]
}
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


{
    "num_found": 1,
    "files": [
        "skeleton__v4__rid-864691135463611454__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz"
    ]
}
TEST PASSED


# Small bulk skeleton request tests
## This routine truncates the request list to a small number (10 at the time of this writing), returns any skeletons that are available, and submits the rest to the asynchronous queue

In [41]:
result = skclient.get_bulk_skeletons(bulk_rids, skeleton_version=skvn, output_format='dict')
# We can't assert both root ids but only one was generated by the previous tests above.
# The other root id will be asyncronously triggered by this test but won't be available for 20-60 seconds afterwards.
assert(str(bulk_rids[0]) in result.keys())
print("TEST PASSED")

result = skclient.get_bulk_skeletons(bulk_rids, skeleton_version=skvn, output_format='dict', verbose_level=1)
# We can't assert both root ids but only one was generated by the previous tests above.
# The other root id will be asyncronously triggered by this test but won't be available for 20-60 seconds afterwards.
assert(str(bulk_rids[0]) in result.keys())
print("TEST PASSED")

result = skclient.get_bulk_skeletons(larger_bulk_rids, skeleton_version=skvn, output_format='dict', verbose_level=1)
# We can't assert both root ids but only one was generated by the previous tests above.
# The other root id will be asyncronously triggered by this test but won't be available for 20-60 seconds afterwards.
assert(str(bulk_rids[0]) in result.keys())
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:Generated skeletons for root_ids [864691135463611454, 864691135687456480] (with generate_missing_skeletons=False)
INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


TEST PASSED


INFO:root:Generated skeletons for root_ids [864691135463611454, 864691135687456480] (with generate_missing_skeletons=False)


TEST PASSED


INFO:root:Queued asynchronous skeleton generation for one batch of root_ids: [864691135463611454, 864691135687456480]
INFO:root:Upper estimate to generate one batch of 2 skeletons: 60.0 seconds
INFO:root:Upper estimate to generate all 2 skeletons: 1.0 minutes


<class 'float'> 60.0
TEST PASSED


INFO:root:Queued asynchronous skeleton generation for one batch of root_ids: [864691135463611454, 864691135687456480]
INFO:root:Upper estimate to generate one batch of 2 skeletons: 60.0 seconds
INFO:root:Upper estimate to generate all 2 skeletons: 1.0 minutes


<class 'float'> 60.0
TEST PASSED


# Asynchronous bulk skeleton request tests
## This routine submits a large number of requests and returns only the estimated time to complete the job; it doesn't return any skeletons.
### The estimated job time depends on the number of parallel workers available on the serverm with each skeleton allocated 60s for estimation purposes.
### For example, with 10 workers, 1–10 skeletons would take 60s, 11–20 skeletons would take 120s, etc.
### At the time of this writing, all servers are configured to use 30 workers.

In [ ]:
result = skclient.generate_bulk_skeletons_async(bulk_rids, skeleton_version=skvn)
print(type(result), result)
assert result == 60.0
print("TEST PASSED")

result = skclient.generate_bulk_skeletons_async(bulk_rids, skeleton_version=skvn, verbose_level=1)
print(type(result), result)
assert result == 60.0
print("TEST PASSED")

result = skclient.generate_bulk_skeletons_async(larger_bulk_rids, skeleton_version=skvn, verbose_level=1)
print(type(result), result)
assert result == 60.0
print("TEST PASSED")

## Meshwork tests
### At the current time, I implemented meshwork generation and caching, but then removed it at the suggestion of other team members, saving the code for possible future use. Consequently, thge meshwork routines can't be tested until the code is added back at some later time. Therefore, the following tests are currently disabled.

In [37]:
RUN_MESHWORK_TESTS = False

In [38]:
%%time
from io import BytesIO

if RUN_MESHWORK_TESTS:
    import pcg_skel
    
    mw_bytes = skclient.get_meshwork(single_rid, datastack_name, verbose_level=1)
    print(len(mw_bytes))
    nrn = pcg_skel.meshwork.load_meshwork(BytesIO(mw_bytes))
    print(nrn)
    print(len(nrn.vertices), len(nrn.edges), len(nrn.anno['pre_syn']), len(nrn.anno['post_syn']))
    print(nrn.distance_to_root(nrn.anno.post_syn.mesh_index) / 1000)

    print("This test doesn't appear to be fully implemented yet, as it doesn't contain an assertion clause.")
    
    print("TEST PASSED")

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 13.1 µs


In [39]:
%%time
if RUN_MESHWORK_TESTS:
    estimated_time = skclient.generate_bulk_meshworks_async(bulk_rids, datastack_name, verbose_level=1)
    print(type(result), result)
    assert result == 60.0
print("TEST PASSED")

CPU times: user 8 µs, sys: 3 µs, total: 11 µs
Wall time: 21 µs
